In [2]:
%pip install open3d

  Obtaining dependency information for open3d from https://files.pythonhosted.org/packages/91/79/7609d49bb8c86d5dcd9684f72449b512d8787708f975fd003acb78c63786/open3d-0.18.0-cp311-cp311-macosx_13_0_arm64.whl.metadata
  Obtaining dependency information for dash>=2.6.0 from https://files.pythonhosted.org/packages/7b/62/438626ab498869249e99376684ba6d88931e7c2dd86d03a92d9d7a9b4c54/dash-2.14.2-py3-none-any.whl.metadata
  Obtaining dependency information for configargparse from https://files.pythonhosted.org/packages/6f/b3/b4ac838711fd74a2b4e6f746703cf9dd2cf5462d17dac07e349234e21b97/ConfigArgParse-1.7-py3-none-any.whl.metadata
  Obtaining dependency information for ansi2html from https://files.pythonhosted.org/packages/42/d7/1bc3433a2406b891182c25a7926abe3eb19af5870d540375ebf7795885b3/ansi2html-1.9.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 8.5 MB/s eta 0:00:0000:01m0:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 34.5 MB/s eta 0:00:00 0:

In [4]:
%pip install mediapipe

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/a3/3a/f309c6bdebe596cc8c960542e167331cb01ef130ec38f3da46a499718889/mediapipe-0.10.9-cp311-cp311-macosx_11_0_universal2.whl.metadata
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/4c/c3/ccff2e1bfe2bb47a7eaebc4280e93bd2f97ebbe5b3573d48bcfcc0c32387/opencv_contrib_python-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 MB 34.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 44.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.4
    Uninstalling protobuf-4.23.4:
      Successfully uninstalled protobuf-4.23.4
Note: you may need to rest

In [1]:
import cv2
from utils_display import DisplayHand
from utils_mediapipe import MediaPipeHand
from utils_joint_angle import GestureRecognition
import numpy as np
import time
import subprocess
import argparse

# ArgumentParser 생성
parser = argparse.ArgumentParser()
parser.add_argument('-m', '--mode', default='eval', help='train / eval')
args, unknown_args = parser.parse_known_args()

# mode 변수 설정
mode = args.mode

# mediapipe hand 클래스 로드
pipe = MediaPipeHand(static_image_mode=False, max_num_hands=4)

# 디스플레이 클래스 로드
disp = DisplayHand(max_num_hands=1)

# 비디오 캡처 시작
cap = cv2.VideoCapture(1)

# 제스처 인식 클래스 로드
gest = GestureRecognition(mode)

# 'home' 제스처가 인식된 시간을 저장하는 변수
home_start_time = None

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    # 3인칭 뷰를 위해 이미지 뒤집기
    img = cv2.flip(img, 1)

    # 이미지를 참조로 전달할 수 없게 설정
    img.flags.writeable = False

    # 피드포워드를 통해 키포인트 추출
    param = pipe.forward(img)

    if (param[0]['class'] is not None) and (mode == 'eval'):
        param[0]['gesture'] = gest.eval(param[0]['angle'])
        
        # 'home' 제스처가 인식되면 음성 출력 및 시간 기록
        if param[0]['gesture'] == 'home':
            if home_start_time is None:
                home_start_time = time.time()
            else:
                elapsed_time = time.time() - home_start_time
                if elapsed_time >= 3:  # 3초 이상 동안 'home' 제스처가 인식되면
                    subprocess.run(["say", "안내를 시작합니다"])  # macOS에서는 say 명령어를 사용하여 TTS 실행
                    home_start_time = None  # 초기화
        else:
            home_start_time = None  # 'home' 제스처가 아닌 경우 초기화

    # 이미지를 참조로 전달할 수 있게 설정
    img.flags.writeable = True

    # 키포인트를 2D로 그려서 표시
    disp.draw2d(img, param)

    # 창에 이미지 표시
    cv2.imshow('img 2D', img)

    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1704950112.105832       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-01-11 14:15:12.298 python[8716:266463] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


: 